In [14]:
# Riot LoL Minimal Flow — Get Match History via Riot ID + Match-V5
# Uses ONLY:
#  - /riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}
#  - /lol/match/v5/matches/by-puuid/{puuid}/ids
# -------------------------------------------------------------

from datetime import datetime, timezone
import requests
import time
import pandas as pd
import os

# =============================
# CONFIG
# =============================
API_KEY = "RGAPI-4ed562c4-7662-4dd1-89d9-1510f8b84c7d"  # <-- replace with your Riot API key
GAME_NAME = "Faker"       # Riot ID name
TAG_LINE = "KR1"          # Riot ID tagline
REGION = "ASIA"           # choose from: AMERICAS, EUROPE, ASIA
QUEUE = 420               # optional queue filter (420 = ranked solo); set to None for all
OUT_DIR = "output_minimal"
os.makedirs(OUT_DIR, exist_ok=True)

SLEEP_BETWEEN_CALLS = 1.2
RETRY_MAX = 5

# =============================
# HELPERS
# =============================
def _headers():
    return {"X-Riot-Token": API_KEY}

def _request_json(url, params=None):
    for attempt in range(RETRY_MAX):
        try:
            r = requests.get(url, headers=_headers(), params=params, timeout=30)
            if r.status_code == 200:
                time.sleep(SLEEP_BETWEEN_CALLS)
                return r.json()
            elif r.status_code in (429, 503):
                delay = float(r.headers.get("Retry-After", 1.5 * (2 ** attempt)))
                print(f"Rate limited, sleeping {delay:.1f}s...")
                time.sleep(delay)
            elif r.status_code in (401, 403):
                raise RuntimeError(f"{r.status_code} {r.text[:200]} → Check API key validity or region access.")
            elif r.status_code == 404:
                return None
            else:
                print(f"HTTP {r.status_code}: {r.text[:120]}")
                time.sleep(2)
        except requests.RequestException as e:
            print("Network error:", e)
            time.sleep(2)
    raise RuntimeError(f"Request failed after {RETRY_MAX} attempts.")

# =============================
# STEP 1 — Resolve Riot ID → PUUID (Account-V1)
# =============================
def get_puuid_from_riotid(gameName, tagLine, region):
    url = f"https://{region.lower()}.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}"
    data = _request_json(url)
    if not data or "puuid" not in data:
        raise RuntimeError(f"Failed to resolve Riot ID {gameName}#{tagLine}. Response: {data}")
    puuid = data["puuid"]
    print(f"✅ {gameName}#{tagLine} → PUUID: {puuid}")
    return puuid

# =============================
# STEP 2 — Get Match IDs (Match-V5)
# =============================
def get_all_match_ids(puuid, region, queue=None):
    base_url = f"https://{region.lower()}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids"
    start = 0
    all_ids = []

    while True:
        params = {"start": start, "count": 100}
        if queue is not None:
            params["queue"] = queue
        ids = _request_json(base_url, params=params)
        if not ids:
            break
        all_ids.extend(ids)
        print(f"Fetched {len(ids)} IDs (total {len(all_ids)})")
        start += len(ids)

    print(f"✅ Total matches found: {len(all_ids)}")
    return all_ids

# =============================
# RUN
# =============================
puuid = get_puuid_from_riotid(GAME_NAME, TAG_LINE, REGION)
match_ids = get_all_match_ids(puuid, REGION, queue=None)

# Save to CSV
out_path = os.path.join(OUT_DIR, f"match_ids_{GAME_NAME}_{TAG_LINE}.csv")
pd.DataFrame({"matchId": match_ids}).to_csv(out_path, index=False)
print("Saved match IDs to:", out_path)

# preview
pd.read_csv(out_path).head()


✅ Faker#KR1 → PUUID: -Hg6wwGXipM9HwIYP5ic0UYMqa9gi89itCH8jPHmcUEUyr9fRJKVTTlF0LPAnz9TBW4g71lEGW2Lhg
Fetched 76 IDs (total 76)
✅ Total matches found: 76
Saved match IDs to: output_minimal/match_ids_Faker_KR1.csv


,matchId
0,KR_7768456553
1,KR_7768428775
2,KR_7737155171
3,KR_7694218699
4,KR_7669256754


In [15]:
# =============================
# STEP 3 — Fetch Details for First 10 Matches
# =============================
import json, os

def get_match_detail(match_id, region):
    """Fetch full match JSON for a given matchId from Match-V5."""
    url = f"https://{region.lower()}.api.riotgames.com/lol/match/v5/matches/{match_id}"
    data = _request_json(url)
    if not data:
        print(f"⚠️ Skipping {match_id} (no data)")
        return None
    return data

# Limit to first 10 matches for testing
sample_match_ids = match_ids[:10]
print(f"Fetching details for {len(sample_match_ids)} matches...")

details = []
for i, mid in enumerate(sample_match_ids, start=1):
    info = get_match_detail(mid, REGION)
    if info:
        details.append(info)
    print(f"  [{i}/{len(sample_match_ids)}] Done {mid}")
    time.sleep(SLEEP_BETWEEN_CALLS)

print(f"✅ Pulled {len(details)} match details")

# Save raw JSON (optional)
out_json = os.path.join(OUT_DIR, f"match_details_sample_{GAME_NAME}_{TAG_LINE}.json")
with open(out_json, "w") as f:
    json.dump(details, f)
print("Saved sample match data to:", out_json)


Fetching details for 10 matches...
  [1/10] Done KR_7768456553
  [2/10] Done KR_7768428775
  [3/10] Done KR_7737155171
  [4/10] Done KR_7694218699
  [5/10] Done KR_7669256754
  [6/10] Done KR_7669237789
  [7/10] Done KR_7669222323
  [8/10] Done KR_7669206621
  [9/10] Done KR_7636932338
  [10/10] Done KR_7599078493
✅ Pulled 10 match details
Saved sample match data to: output_minimal/match_details_sample_Faker_KR1.json


In [16]:
# =============================
# STEP 4 — Fetch Match Timelines (Events / Gold / XP)
# =============================
import json, os

def get_match_timeline(match_id, region):
    """Fetch timeline JSON for a given matchId."""
    url = f"https://{region.lower()}.api.riotgames.com/lol/match/v5/matches/{match_id}/timeline"
    data = _request_json(url)
    if not data:
        print(f"⚠️ Skipping timeline for {match_id}")
        return None
    return data

# Limit to first 3–5 matches for testing (timeline is large)
timeline_match_ids = match_ids[:5]
timelines = []

for i, mid in enumerate(timeline_match_ids, start=1):
    timeline = get_match_timeline(mid, REGION)
    if timeline:
        timelines.append(timeline)
    print(f"  [{i}/{len(timeline_match_ids)}] timeline done for {mid}")
    time.sleep(SLEEP_BETWEEN_CALLS)

print(f"✅ Pulled {len(timelines)} match timelines")

# Save raw JSON
out_timeline_json = os.path.join(OUT_DIR, f"match_timelines_sample_{GAME_NAME}_{TAG_LINE}.json")
with open(out_timeline_json, "w") as f:
    json.dump(timelines, f)
print("Saved timeline data to:", out_timeline_json)


  [1/5] timeline done for KR_7768456553
  [2/5] timeline done for KR_7768428775
  [3/5] timeline done for KR_7737155171
  [4/5] timeline done for KR_7694218699
  [5/5] timeline done for KR_7669256754
✅ Pulled 5 match timelines
Saved timeline data to: output_minimal/match_timelines_sample_Faker_KR1.json


In [18]:
timeline_rows = []
for tl in timelines:
    match_id = tl["metadata"]["matchId"]
    for frame in tl["info"]["frames"]:
        tstamp = frame["timestamp"] // 60000  # minutes
        for pid, pdata in frame["participantFrames"].items():
            timeline_rows.append({
                "matchId": match_id,
                "minute": tstamp,
                "participantId": pid,
                "totalGold": pdata["totalGold"],
                "xp": pdata["xp"],
                "level": pdata["level"],
            })
timeline_df = pd.DataFrame(timeline_rows)
timeline_df


,matchId,minute,participantId,totalGold,xp,level
0,KR_7768456553,0,1,500,0,1
1,KR_7768456553,0,2,500,0,1
2,KR_7768456553,0,3,500,0,1
3,KR_7768456553,0,4,500,0,1
4,KR_7768456553,0,5,500,0,1
...,...,...,...,...,...,...
1005,KR_7669256754,23,6,17067,28684,18
1006,KR_7669256754,23,7,14813,26515,18
1007,KR_7669256754,23,8,16864,27554,18
1008,KR_7669256754,23,9,18826,30208,18


In [20]:
player_df = timeline_df[timeline_df['participantId'] == "1"]
player_df

,matchId,minute,participantId,totalGold,xp,level
0,KR_7768456553,0,1,500,0,1
10,KR_7768456553,1,1,905,18,1
20,KR_7768456553,2,1,1073,267,1
30,KR_7768456553,3,1,1993,915,3
40,KR_7768456553,4,1,3015,1556,4
...,...,...,...,...,...,...
960,KR_7669256754,20,1,14716,25973,18
970,KR_7669256754,21,1,15121,26615,18
980,KR_7669256754,22,1,15796,28492,18
990,KR_7669256754,23,1,16156,28792,18


In [10]:
# Flatten participant data into a table
participants = []
for match in details:
    info = match["info"]
    for p in info["participants"]:
        participants.append({
            "matchId": info["gameId"],
            "gameDuration": info["gameDuration"],
            "queueId": info["queueId"],
            "championName": p["championName"],
            "kills": p["kills"],
            "deaths": p["deaths"],
            "assists": p["assists"],
            "win": p["win"],
            "summonerName": p["summonerName"],
            "teamId": p["teamId"],
            "role": p["role"],
            "lane": p["lane"]
        })

df = pd.DataFrame(participants)
print("Participants shape:", df.shape)
df


Participants shape: (106, 12)


,matchId,gameDuration,queueId,championName,kills,deaths,assists,win,summonerName,teamId,role,lane
0,5403251060,1690,420,Rumble,4,6,8,True,,100,SOLO,TOP
1,5403251060,1690,420,Ambessa,13,5,9,True,,100,NONE,JUNGLE
2,5403251060,1690,420,Ahri,4,6,11,True,,100,SOLO,MIDDLE
3,5403251060,1690,420,Riven,14,3,10,True,,100,SOLO,BOTTOM
4,5403251060,1690,420,Blitzcrank,2,3,25,True,,100,NONE,JUNGLE
...,...,...,...,...,...,...,...,...,...,...,...,...
101,5402460752,1096,420,Gwen,5,1,3,True,,200,SUPPORT,NONE
102,5402460752,1096,420,Khazix,10,0,1,True,,200,SUPPORT,NONE
103,5402460752,1096,420,Ahri,4,0,7,True,,200,SUPPORT,NONE
104,5402460752,1096,420,Smolder,2,3,3,True,,200,SUPPORT,NONE


In [11]:
df_match = df[df['matchId'] == 5403251060]
df_match

,matchId,gameDuration,queueId,championName,kills,deaths,assists,win,summonerName,teamId,role,lane
0,5403251060,1690,420,Rumble,4,6,8,True,,100,SOLO,TOP
1,5403251060,1690,420,Ambessa,13,5,9,True,,100,NONE,JUNGLE
2,5403251060,1690,420,Ahri,4,6,11,True,,100,SOLO,MIDDLE
3,5403251060,1690,420,Riven,14,3,10,True,,100,SOLO,BOTTOM
4,5403251060,1690,420,Blitzcrank,2,3,25,True,,100,NONE,JUNGLE
5,5403251060,1690,420,Nidalee,4,6,10,False,,200,SOLO,TOP
6,5403251060,1690,420,JarvanIV,11,7,5,False,,200,NONE,JUNGLE
7,5403251060,1690,420,Mel,7,7,2,False,,200,SOLO,MIDDLE
8,5403251060,1690,420,Jhin,0,8,11,False,,200,CARRY,BOTTOM
9,5403251060,1690,420,Soraka,1,9,5,False,,200,SUPPORT,BOTTOM
